# New York Cab Ride Duration Prediction

In [9]:
# Importing Library
import featuretools as ft
import utils
from utils import load_nyc_taxi_data, compute_features, preview, feature_importances
from sklearn.ensemble import GradientBoostingRegressor
from featuretools.primitives import (Weekend, Minute, Hour, Day, Week, Month,
                                     Weekday, Weekend, Count, Sum, Mean, Median, Std, Min, Max)
import numpy as np
ft.__version__
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Preparation

In [13]:
trips, pickup_neighborhoods, dropoff_neighborhoods = load_nyc_taxi_data()
preview(trips, 10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,trip_duration,pickup_neighborhood,dropoff_neighborhood
0,0,2,2016-01-01 00:00:19,2016-01-01 00:06:31,3,1.32,-73.961258,40.796200,-73.950050,40.787312,2,372.0,AH,C
672146,672146,1,2016-04-29 07:01:31,2016-04-29 07:15:46,1,3.30,-73.949951,40.784653,-73.982536,40.755470,1,855.0,C,AA
672147,672147,2,2016-04-29 07:01:43,2016-04-29 07:09:15,1,1.14,-73.967331,40.757370,-73.954277,40.765282,1,452.0,N,K
672148,672148,1,2016-04-29 07:01:46,2016-04-29 07:07:54,1,1.10,-74.003082,40.727509,-73.984703,40.724377,1,368.0,AB,AC
672149,672149,2,2016-04-29 07:01:46,2016-04-29 07:06:48,2,1.40,-73.990158,40.772350,-73.982147,40.759800,1,302.0,AR,AA
672150,672150,1,2016-04-29 07:01:59,2016-04-29 07:07:33,1,1.20,-73.983681,40.746677,-73.971703,40.762463,2,334.0,AO,A
672151,672151,2,2016-04-29 07:02:11,2016-04-29 07:15:24,2,2.13,-73.994209,40.750999,-73.969391,40.761539,1,793.0,D,AK
672152,672152,1,2016-04-29 07:02:11,2016-04-29 07:06:44,1,1.00,-73.983276,40.770985,-73.980110,40.760666,1,273.0,AR,A
672153,672153,2,2016-04-29 07:02:13,2016-04-29 07:08:36,1,1.17,-73.980141,40.743168,-73.983391,40.754665,1,383.0,Y,AA
672154,672154,1,2016-04-29 07:02:16,2016-04-29 07:04:07,1,0.50,-73.965973,40.765381,-73.970558,40.758724,1,111.0,AK,N


In [14]:
entities = {
        "trips": (trips, "id", 'pickup_datetime' ),
        "pickup_neighborhoods": (pickup_neighborhoods, "neighborhood_id"),
        "dropoff_neighborhoods": (dropoff_neighborhoods, "neighborhood_id"),
        }

relationships = [("pickup_neighborhoods", "neighborhood_id", "trips", "pickup_neighborhood"),
                 ("dropoff_neighborhoods", "neighborhood_id", "trips", "dropoff_neighborhood")]

### Cutoff time for each instance of the target_entity

In [15]:
# choose to only select trips that started after January 12th, 2016
cutoff_time = trips[['id', 'pickup_datetime']]
cutoff_time = cutoff_time[cutoff_time['pickup_datetime'] > "2016-01-12"]
preview(cutoff_time, 10)

,id,pickup_datetime
56311,56311,2016-01-12 00:00:25
698765,698765,2016-05-03 18:54:53
698766,698766,2016-05-03 18:55:37
698767,698767,2016-05-03 18:55:38
698768,698768,2016-05-03 18:55:49
698769,698769,2016-05-03 18:55:58
698770,698770,2016-05-03 18:56:22
698771,698771,2016-05-03 18:56:24
698772,698772,2016-05-03 18:56:51
698773,698773,2016-05-03 18:56:56


## Create baseline features using Deep Feature Synthesis
### Create transform features using transform primitives

In [16]:
trans_primitives = [Weekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [17]:
# Printing Features Properties
print "Number of features: %d" % len(features)
features

Number of features: 13


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: dropoff_neighborhood>,
 <Feature: payment_type>,
 <Feature: pickup_neighborhood>,
 <Feature: trip_duration>,
 <Feature: trip_distance>,
 <Feature: dropoff_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>]

In [18]:
# Computing The Features
feature_matrix = compute_features(features, cutoff_time)

Progress: 100%|█████████████████████████| 5/5 [01:31<00:00, 18.28s/cutoff time]
Finishing computing...


In [19]:
# Preview Features Created
preview(feature_matrix, 5)

,IS_WEEKEND(dropoff_datetime),trip_distance,passenger_count,dropoff_neighborhood = AD,dropoff_neighborhood = A,dropoff_neighborhood = AA,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,...,pickup_neighborhood = AD,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,pickup_neighborhood = AR,pickup_neighborhood = AK,pickup_neighborhood = AO,pickup_neighborhood = N,pickup_neighborhood = R,pickup_neighborhood = O
id,,,,,,,,,,,,,,,,,,,,,
56311,False,1.61,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
691284,False,0.61,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
691285,False,0.88,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
691286,False,1.90,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
691288,False,1.00,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


# Building The Model

In [20]:
# separates the whole feature matrix into train data feature matrix, 
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [21]:
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train, y_train)
model.score(X_test, y_test)

      Iter       Train Loss   Remaining Time 
         1           0.4925            8.13m
         2           0.4333            8.24m
         3           0.3843            7.89m
         4           0.3446            7.82m
         5           0.3119            7.75m
         6           0.2852            7.60m
         7           0.2634            7.46m
         8           0.2454            7.36m
         9           0.2305            7.25m
        10           0.2183            7.16m
        20           0.1666            6.40m
        30           0.1558            5.46m
        40           0.1514            4.62m
        50           0.1488            3.77m
        60           0.1472            3.03m
        70           0.1458            2.28m
        80           0.1448            1.52m
        90           0.1440           45.80s
       100           0.1433            0.00s


0.72200175704571445

## Adding more Transform Primitives

In [22]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, Weekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [23]:
print "Number of features: %d" % len(features)
features

Number of features: 25


[<Feature: passenger_count>,
 <Feature: dropoff_neighborhood>,
 <Feature: payment_type>,
 <Feature: vendor_id>,
 <Feature: pickup_neighborhood>,
 <Feature: trip_duration>,
 <Feature: trip_distance>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: dropoff_neighborhoods.longitude>,
 <Feature: IS_WEEKEND(dropoff_datetime)>]

In [24]:
# Computing Features
feature_matrix = compute_features(features, cutoff_time)

Progress: 100%|█████████████████████████| 5/5 [01:35<00:00, 19.18s/cutoff time]
Finishing computing...


In [25]:
# Feature Preview
preview(feature_matrix, 10)

,WEEKDAY(dropoff_datetime),dropoff_neighborhoods.latitude,MINUTE(dropoff_datetime),WEEK(dropoff_datetime),passenger_count,trip_duration,HOUR(pickup_datetime),pickup_neighborhoods.latitude,vendor_id,dropoff_neighborhoods.longitude,...,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,dropoff_neighborhood = N,dropoff_neighborhood = AO,dropoff_neighborhood = AK,HOUR(dropoff_datetime),IS_WEEKEND(dropoff_datetime),trip_distance
id,,,,,,,,,,,,,,,,,,,,,
56311,1,40.721435,11,2,1,645.0,0,40.720245,2,-73.998366,...,0,0,0,0,0,0,0,0,False,1.61
691284,0,40.721435,24,18,2,160.0,12,40.729652,2,-73.998366,...,0,0,0,0,0,0,0,12,False,0.61
691285,0,40.785005,27,18,2,295.0,12,40.776270,2,-73.976050,...,0,0,0,0,0,0,0,12,False,0.88
691286,0,40.757707,48,18,1,1573.0,12,40.742531,1,-73.986446,...,0,0,0,0,0,0,0,12,False,1.90
691288,0,40.761087,30,18,1,404.0,12,40.747126,1,-73.995736,...,0,0,0,0,0,0,0,12,False,1.00
691289,0,40.761492,55,18,1,1906.0,12,40.721435,2,-73.975899,...,0,0,0,0,0,0,0,12,False,3.24
691290,0,40.764723,26,18,1,156.0,12,40.764723,1,-73.966696,...,0,0,0,0,0,0,1,12,False,0.10
691291,0,40.776270,37,18,1,827.0,12,40.766809,1,-73.982322,...,0,0,0,0,0,0,0,12,False,1.60
691292,0,40.764723,39,18,1,883.0,12,40.752186,1,-73.966696,...,0,0,0,0,0,0,1,12,False,1.50


# Building a New Model

In [26]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [27]:
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train,y_train)
model.score(X_test,y_test)

      Iter       Train Loss   Remaining Time 
         1           0.4925           12.94m
         2           0.4333           12.66m
         3           0.3843           11.79m
         4           0.3444           11.37m
         5           0.3117           11.39m
         6           0.2848           11.48m
         7           0.2620           11.38m
         8           0.2435           11.13m
         9           0.2282           10.84m
        10           0.2152           10.68m
        20           0.1588            9.57m
        30           0.1415            8.24m
        40           0.1332            7.08m
        50           0.1283            5.81m
        60           0.1252            4.60m
        70           0.1227            3.42m
        80           0.1207            2.29m
        90           0.1191            1.12m
       100           0.1177            0.00s


0.7755608981558122

# Adding Aggregation Primitives
## It will features for parent entities and then add them to prediction entity which is trips entity

In [28]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, Weekend]
aggregation_primitives = [Count, Sum, Mean, Median, Std, Max, Min]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=aggregation_primitives,
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [29]:
print "Number of features: %d" % len(features)
features

Number of features: 63


[<Feature: passenger_count>,
 <Feature: dropoff_neighborhood>,
 <Feature: payment_type>,
 <Feature: vendor_id>,
 <Feature: pickup_neighborhood>,
 <Feature: trip_duration>,
 <Feature: trip_distance>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.longitude>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: dropoff_neighborhoods.SUM(trips.trip_duration)>,
 <Feature: pickup_neighborhoods.STD(trips.trip

In [30]:
feature_matrix = compute_features(features, cutoff_time)

Progress: 100%|█████████████████████████| 5/5 [03:25<00:00, 41.00s/cutoff time]
Finishing computing...


In [31]:
preview(feature_matrix, 10)

,dropoff_neighborhoods.MAX(trips.trip_duration),pickup_neighborhoods.MEDIAN(trips.passenger_count),pickup_neighborhoods.MEDIAN(trips.trip_distance),HOUR(dropoff_datetime),dropoff_neighborhoods.COUNT(trips),DAY(pickup_datetime),pickup_neighborhoods.latitude,pickup_neighborhoods.SUM(trips.passenger_count),pickup_neighborhoods.STD(trips.trip_distance),dropoff_neighborhoods.SUM(trips.passenger_count),...,pickup_neighborhoods.STD(trips.passenger_count),MONTH(pickup_datetime),pickup_neighborhoods.MEAN(trips.trip_duration),WEEK(pickup_datetime),dropoff_neighborhoods.MEAN(trips.trip_distance),MONTH(dropoff_datetime),payment_type,MINUTE(dropoff_datetime),WEEK(dropoff_datetime),dropoff_neighborhoods.MAX(trips.passenger_count)
id,,,,,,,,,,,,,,,,,,,,,
56311,3572.0,1.0,2.40,0,1396.0,12,40.720245,2283.0,2.517060,2375.0,...,1.331649,1,740.870871,2,2.495358,1,1,11,2,6.0
691284,3603.0,1.0,1.60,12,16736.0,2,40.729652,34521.0,2.099009,28154.0,...,1.310235,5,753.813680,18,2.338798,5,1,24,18,6.0
691285,3602.0,1.0,1.60,12,19017.0,2,40.776270,36299.0,2.111243,31836.0,...,1.315396,5,681.405688,18,2.176976,5,1,27,18,6.0
691286,3606.0,1.0,1.49,12,28805.0,2,40.742531,31158.0,2.137177,49208.0,...,1.330198,5,682.624440,18,2.365290,5,1,48,18,6.0
691288,3580.0,1.0,1.40,12,16985.0,2,40.747126,43543.0,2.382449,28197.0,...,1.319326,5,714.648716,18,2.067381,5,1,30,18,6.0
691289,3606.0,1.0,1.90,12,31541.0,2,40.721435,30913.0,4.278882,52591.0,...,1.315238,5,818.141251,18,2.102551,5,1,55,18,6.0
691290,3580.0,1.0,1.30,12,21894.0,2,40.764723,43212.0,1.846378,36175.0,...,1.315462,5,637.726834,18,1.732215,5,2,26,18,6.0
691291,3604.0,1.0,1.63,12,21272.0,2,40.766809,32656.0,2.206183,35282.0,...,1.332742,5,707.024093,18,2.061938,5,1,37,18,6.0
691292,3580.0,1.0,1.49,12,21894.0,2,40.752186,57862.0,2.488034,36175.0,...,1.306561,5,749.696305,18,1.732215,5,1,39,18,6.0


# Building a New Model

In [32]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [33]:
# note: this may take up to 30 minutes to run
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.4925           69.57m
         2           0.4333           59.03m
         3           0.3843           52.16m
         4           0.3444           46.15m
         5           0.3117           42.86m
         6           0.2848           41.78m
         7           0.2620           41.13m
         8           0.2435           39.79m
         9           0.2282           39.09m
        10           0.2152           38.86m
        20           0.1585           33.15m
        30           0.1420           27.14m
        40           0.1332           22.12m
        50           0.1271           17.79m
        60           0.1238           13.76m
        70           0.1211           10.02m
        80           0.1191            6.52m
        90           0.1176            3.19m
       100           0.1163            0.00s


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=True, warm_start=False)

# Evaluate on Test Data

In [34]:
model.score(X_test,y_test)

0.77808885638984859

# We can also make predictions using our model

In [35]:
y_pred = model.predict(X_test)
y_pred = np.exp(y_pred) - 1 # undo the log we took earlier
y_pred[5:]

array([  557.67992664,   590.2602792 ,  1497.39684679, ...,  1063.48382696,
        1800.89361932,   739.60249439])

# Additional Analysis

### Let's look at how important each feature was for the model

In [36]:
feature_importances(model, feature_matrix.columns, n=15)

1: Feature: trip_distance, 0.314
2: Feature: HOUR(pickup_datetime), 0.126
3: Feature: HOUR(dropoff_datetime), 0.089
4: Feature: WEEKDAY(pickup_datetime), 0.052
5: Feature: dropoff_neighborhoods.latitude, 0.046
6: Feature: dropoff_neighborhoods.longitude, 0.036
7: Feature: dropoff_neighborhoods.STD(trips.trip_distance), 0.027
8: Feature: dropoff_neighborhoods.MIN(trips.passenger_count), 0.022
9: Feature: dropoff_neighborhoods.MEDIAN(trips.trip_duration), 0.022
10: Feature: pickup_neighborhoods.MEDIAN(trips.trip_distance), 0.021
11: Feature: IS_WEEKEND(pickup_datetime), 0.021
12: Feature: WEEKDAY(dropoff_datetime), 0.020
13: Feature: WEEK(pickup_datetime), 0.019
14: Feature: dropoff_neighborhoods.MEAN(trips.trip_duration), 0.019
15: Feature: MONTH(dropoff_datetime), 0.018
